In [ ]:
!pip install vit_keras
!pip install tensorflow-addons
!pip install openpyxl
!pip install opencv-python opencv-python-headless

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 18.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from vit_keras import vit
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import cv2
import re

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
vit_model = vit.vit_b32(
    image_size=(224,224),
    activation='softmax',
    pretrained=True,
    include_top=False,
    pretrained_top=False,
    classes=None
)

353253686/353253686 [==============================] - 1s 0us/step


/usr/local/lib/python3.10/dist-packages/vit_keras/utils.py:81: UserWarning: Resizing position embeddings from 12, 12 to 7, 7
  warnings.warn(


In [ ]:
# Initialize lists to hold images, numerical features, and labels
images, numerical_features, labels = [], [], []

numerical_features_df = pd.read_excel("/content/drive/MyDrive/Baseline_DR/DR_Baseline_FINAL.xlsx")
occurances0 = (numerical_features_df["os"]==0).sum() + (numerical_features_df["od"]==0).sum()
occurances1 = (numerical_features_df["os"]==1).sum() + (numerical_features_df["od"]==1).sum()
occurances2 = (numerical_features_df["os"]==2).sum() + (numerical_features_df["od"]==2).sum()
occurances3 = (numerical_features_df["os"]==3).sum() + (numerical_features_df["od"]==3).sum()
occurances4 = (numerical_features_df["os"]==4).sum() + (numerical_features_df["od"]==4).sum()
print(occurances0)
print(occurances1)
print(occurances2)
print(occurances3)
print(occurances4)

1856
175
115
23
29


In [ ]:
def extract_capital_alphabets_and_replace_underscores(input_string):

    initial_chars = "D R P E "
    if input_string.startswith(initial_chars):
        input_string = input_string[len(initial_chars):]
    # Use regular expression to match capital alphabets and underscores
    pattern = r'[A-Z_ ]'

    # Find all matches in the input string
    matches = re.findall(pattern, input_string)

    # print(matches)

    # Concatenate the matches into a single string
    result_string = ''.join(matches)

    # Replace underscores with spaces
    result_string = result_string.replace('_', ' ')

    result_string = result_string.strip()

    return result_string

In [ ]:
count = 0

# Load images and labels
for label in range(5):  # Assuming labels 0 to 4
    folder_path = os.path.join("/content/drive/MyDrive/Baseline_DR", str(label))
    for filename in os.listdir(folder_path):
        count+=1
        if filename.endswith(".jpg"):
            name_key = extract_capital_alphabets_and_replace_underscores(filename)  # Extract name key using the defined function
            # print(filename)
            # print(name_key)
            if name_key:
                num_feature = numerical_features_df[numerical_features_df["names"].str.upper() == name_key]
                # if not num_feature.empty:
                #     for index, row in num_feature.iterrows():
                #         if label == row["DR multiclass"]:
                #             count += 1
                if not num_feature.empty:
                    img_path = os.path.join(folder_path, filename)
                    img = cv2.imread(img_path)
                    img = cv2.resize(img, (224, 224))  # Assuming image_input_shape is (224, 224, 3)
                    img = img / 255.0  # Normalize pixel values

                    images.append(img)
                    numerical_features.append(num_feature.drop(columns=["names", "DR", "DR multiclass", "od", "os"]).values[0])
                    labels.append(label)
print(count)

2198


In [ ]:
print(len(images))
print(len(numerical_features))
print(len(labels))

2116
2116
2116


In [ ]:
###DEFINING THE TRANSFORMS

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of ImageDataGenerator with desired augmentation parameters
datagen1 = ImageDataGenerator(
    rotation_range=15,  # Rotate images by up to 15 degrees
    zoom_range=0.1,  # Zoom images by up to 10%
    brightness_range=[0.8, 1.2],  # Adjust brightness by up to 20%
    horizontal_flip=True,  # Flip images horizontally
    vertical_flip=True,  # Flip images vertically
    fill_mode='nearest'  # Fill in missing pixels with the nearest value
)

# Create an instance of ImageDataGenerator with desired augmentation parameters
datagen2 = ImageDataGenerator(
    rotation_range=15,  # Rotate images by up to 15 degrees
    zoom_range=0.1,  # Zoom images by up to 10%
    brightness_range=[0.8, 1.2],  # Adjust brightness by up to 20%
    horizontal_flip=True,  # Flip images horizontally
    vertical_flip=False,  # Flip images vertically
    fill_mode='nearest'  # Fill in missing pixels with the nearest value
)

# Create an instance of ImageDataGenerator with desired augmentation parameters
datagen3 = ImageDataGenerator(
    rotation_range=15,  # Rotate images by up to 15 degrees
    zoom_range=0.1,  # Zoom images by up to 10%
    brightness_range=[0.8, 1.2],  # Adjust brightness by up to 20%
    horizontal_flip=True,  # Flip images horizontally
    vertical_flip=False,  # Flip images vertically
    fill_mode='nearest'  # Fill in missing pixels with the nearest value
)

# Create an instance of ImageDataGenerator with desired augmentation parameters
datagen4 = ImageDataGenerator(
    rotation_range=15,  # Rotate images by up to 15 degrees
    zoom_range=0.1,  # Zoom images by up to 10%
    brightness_range=[0.8, 1.2],  # Adjust brightness by up to 20%
    horizontal_flip=True,  # Flip images horizontally
    vertical_flip=False,  # Flip images vertically
    fill_mode='nearest'  # Fill in missing pixels with the nearest value
)

In [ ]:
counts = [0] * 5
print(len(labels))
for label in labels:
  counts[label]+=1
print(counts)

2116
[1740, 198, 124, 22, 32]


In [ ]:
import math
# Iterate through your images and apply augmentation
augmented_images = []
augmented_labels = []
augmented_nfs = []


for image, label, nf in zip(images, labels, numerical_features):
    # Reshape the image to (1, height, width, channels) for compatibility with ImageDataGenerator
    image = image.reshape((1,) + image.shape)
    count = 0
    if label == 0:
        count = 1
    else:
        count = 2*counts[0]/counts[label]
        count = math.floor(count) - 1

    while count > 0:  # Use a while loop to apply augmentation 'count' times
        if count % 4 == 0:
            # Generate augmented images from the original image
            augmented_data = datagen1.flow(image, batch_size=1)
        elif (count - 1) % 4 == 0:
            # Generate augmented images from the original image
            augmented_data = datagen2.flow(image, batch_size=1)
        elif (count - 2) % 4 == 0:
            # Generate augmented images from the original image
            augmented_data = datagen3.flow(image, batch_size=1)
        else:
            # Generate augmented images from the original image
            augmented_data = datagen4.flow(image, batch_size=1)

        # Extract augmented image and label
        augmented_image = next(augmented_data)[0].astype('uint8')  # Convert back to uint8 format
        augmented_label = label  # Labels remain unchanged in augmentation
        augmented_nf = nf

        augmented_images.append(augmented_image)
        augmented_labels.append(augmented_label)
        augmented_nfs.append(augmented_nf)
        count -= 1


In [ ]:
label_count = [0] * 5
for label in augmented_labels:
    label_count[label]+=1
print(label_count)

[1740, 3168, 3348, 3454, 3424]


In [ ]:
images += augmented_images
labels += augmented_labels
numerical_features += augmented_nfs

In [ ]:
label_count = [0] * 5
for label in labels:
    label_count[label]+=1
print(label_count)

print(len(images))
print(len(labels))

[3480, 3366, 3472, 3476, 3456]
17250
17250


In [ ]:
image_input_shape = (224, 224, 3)  # Assuming RGB images
num_features = 47
num_classes = 5
learning_rate = 0.001


# Define inputs for image and numerical features
image_input = layers.Input(shape=image_input_shape, name="image_input")
num_features_input = layers.Input(shape=(num_features,), name="num_features_input")

# Obtain image embeddings using ViT
image_embeddings = vit_model(image_input)

print(type(image_embeddings))
print(image_embeddings.shape) #768


# Concatenate image embeddings with numerical features
concatenated_features = layers.Concatenate()([image_embeddings, num_features_input])

# Add fully connected layers for classification
x = layers.Dense(818, activation="relu")(concatenated_features)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation="relu")(x)
output = layers.Dense(num_classes, activation="softmax")(x)

# Create the model
model = Model(inputs=[image_input, num_features_input], outputs=output)

# Create an Adam optimizer with the specified learning rate
adam_optimizer = Adam(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=adam_optimizer,
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# Print model summary
model.summary()

<class 'keras.src.engine.keras_tensor.KerasTensor'>
(None, 768)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_input (InputLayer)    [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 vit-b32 (Functional)        (None, 768)                  8745523   ['image_input[0][0]']         
                                                          2                                       
                                                                                                  
 num_features_input (InputL  [(None, 47)]                 0         []                            
 ayer)                                                                                            
                              

In [ ]:
# Convert lists to numpy arrays
images = np.array(images)
numerical_features = np.array(numerical_features)
labels = np.array(labels)

print(numerical_features.shape)

(17250, 47)


In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Split the data into training and validation sets
images_train, images_val, num_features_train, num_features_val, labels_train, labels_val = train_test_split(images, numerical_features, labels, test_size=0.2, random_state=42)

# Define early stopping criteria
early_stopping_monitor = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)

# Define checkpoint to save model weights
checkpoint = ModelCheckpoint(filepath='/content/drive/My Drive/model_weights_epoch_{epoch:02d}.h5', monitor='val_loss', save_best_only=False, save_weights_only=True, mode='auto', period=1)

# Train your model with early stopping and model checkpoint
history = model.fit(
    [images_train, num_features_train],
    labels_train,
    batch_size=32,
    epochs=50,
    validation_data=([images_val, num_features_val], labels_val),
    callbacks=[early_stopping_monitor, checkpoint]
)


Epoch 1/50
432/432 [==============================] - 802s 2s/step - loss: 4.0669 - accuracy: 0.3896 - val_loss: 1.1482 - val_accuracy: 0.5371
Epoch 2/50
432/432 [==============================] - 762s 2s/step - loss: 1.1645 - accuracy: 0.5308 - val_loss: 0.9173 - val_accuracy: 0.6261
Epoch 3/50
432/432 [==============================] - 761s 2s/step - loss: 1.0017 - accuracy: 0.5979 - val_loss: 0.8154 - val_accuracy: 0.6646
Epoch 4/50
432/432 [==============================] - 759s 2s/step - loss: 0.8997 - accuracy: 0.6438 - val_loss: 0.7047 - val_accuracy: 0.7145
Epoch 5/50
432/432 [==============================] - 758s 2s/step - loss: 0.8362 - accuracy: 0.6722 - val_loss: 0.6573 - val_accuracy: 0.7528
Epoch 6/50
432/432 [==============================] - 760s 2s/step - loss: 0.7515 - accuracy: 0.7010 - val_loss: 0.6078 - val_accuracy: 0.7713
Epoch 7/50
432/432 [==============================] - 761s 2s/step - loss: 0.7433 - accuracy: 0.7086 - val_loss: 0.5886 - val_accuracy: 0.7849

In [ ]:
print(os.getcwd())

In [ ]:
model.save('/content/my_model.h5')

In [ ]:
for label in range(5):  # Assuming labels 0 to 4
    folder_path = os.path.join("/content/drive/MyDrive/Baseline_DR", str(label))
    files = 0
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
          files+=1
    print(label)
    print(files)

In [ ]:
# Plotting the loss curve
plt.figure(figsize=(8, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Iteration Curve')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Make predictions on the validation data
predictions = model.predict([validation_images, validation_numerical_features])

# Calculate AUC
from sklearn.metrics import roc_auc_score

# Assuming labels are binary (0 or 1)
auc = roc_auc_score(validation_labels, predictions)

print("AUC:", auc)

In [ ]:
# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(validation_labels, predictions)

# Calculate specificity and sensitivity
specificity = 1 - fpr
sensitivity = tpr

# Plot specificity vs sensitivity curve
plt.plot(specificity, sensitivity, marker='o', linestyle='-')
plt.xlabel('Specificity')
plt.ylabel('Sensitivity')
plt.title('Specificity vs Sensitivity Curve')
plt.grid(True)
plt.show()
